## Multi Layer Perceptron

- Deep Dense layers
- Activation 계층을 추가하므로써 비선형성을 만드는 것.
- tensorflow package에 logit(softmax 이전 상태, 출력 계층에 z말고 a)을 이용하는 함수를 이용해보고자 함.
- 개념적으로 볼 때 logistic regression(classification이 더 맞는 말)과의 차이는 depth라고 나도 생각됨.

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
print(tf.__version__)

1.15.0


In [4]:
mnist = input_data.read_data_sets("data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [14]:
n_hidden_1 = 256
n_hidden_2 = 128
n_hidden_3 = 64
n_input = 784
n_classes = 10
stddev = 0.1


In [21]:
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

weights = {
    "h1": tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=stddev)),
    "h2": tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=stddev)),
    "h3": tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3], stddev=stddev)),
    "out": tf.Variable(tf.random_normal([n_hidden_3, n_classes], stddev=stddev))
}
biases = {
    "b1": tf.Variable(tf.random_normal([n_hidden_1])),
    "b2": tf.Variable(tf.random_normal([n_hidden_2])),
    "b3": tf.Variable(tf.random_normal([n_hidden_3])),
    "out": tf.Variable(tf.random_normal([n_classes]))
}

In [22]:
def multilayer_perceptron(_X, _weights, _biases):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(_X, _weights["h1"]), _biases["b1"]))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, _weights["h2"]), _biases["b2"]))
    layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, _weights["h3"]), _biases["b3"]))
    return tf.matmul(layer_3, _weights["out"]) + _biases["out"]

In [23]:
pred = multilayer_perceptron(x, weights, biases)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
corr = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(corr, "float"))

In [24]:
max_epochs = 20
batch_size = 100
display_step = 4

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(max_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples / batch_size)
        batch_xs = []; batch_ys = []; feeds = dict()
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            feeds = {x: batch_xs, y: batch_ys}
            sess.run(optimizer, feed_dict=feeds)
            avg_cost += sess.run(cost, feed_dict=feeds)
        avg_cost /= total_batch

        if (epoch + 1) % display_step == 0:
            print("Epoch: %03d/%03d cost: %.9f" % (epoch, max_epochs, avg_cost))
            train_acc = sess.run(accuracy, feed_dict=feeds)
            print("Train Accuracy: %.3f" % train_acc)
            feeds = {x: mnist.test.images, y: mnist.test.labels}
            test_acc = sess.run(accuracy, feed_dict=feeds)
            print("Test Accuracy: %.3f" % test_acc)
print("Optimization finished !")

Epoch: 003/020 cost: 0.109280802
Train Accuracy: 0.970
Test Accuracy: 0.964
Epoch: 007/020 cost: 0.040693321
Train Accuracy: 0.970
Test Accuracy: 0.975
Epoch: 011/020 cost: 0.015786796
Train Accuracy: 1.000
Test Accuracy: 0.976
Epoch: 015/020 cost: 0.006636053
Train Accuracy: 1.000
Test Accuracy: 0.979
Epoch: 019/020 cost: 0.003516116
Train Accuracy: 1.000
Test Accuracy: 0.975
Optimization finished !


logistic regression에서 test accuracy가 92% 정도였던 걸 생각하면<br />
MLP에서 test accuracy가 98%가 되었으니 성능 향상을 보임.

현재 모델은 원래 hidden이 2층이었던 것에서 3층으로 바꾼 건데<br />
아마 deep이 무조건 모델 성능 향상에 도움을 주는 것은 아니라는 식을 끝내고 싶으셨던 것 같다.